# The Indefinite Helmholtz Equation

Similar to our previous example, the first step is to import the respective EvoStencils modules.

In [1]:
from evostencils.optimization.program import Optimizer
from evostencils.code_generation.exastencils import ProgramGenerator
import os
import sys

## Problem Formulation

As a more interesting test case than the one used in our basic tutorial we consider an example of the indefinite Helmholtz equation, as given by

\begin{equation}
	\begin{split}
		(-\nabla ^{2} - k^{2}) u & = f \quad \text{in} \; \left( 0, 1 \right)^2 \\
		u & = 0 \quad \text{on} \; \left( 0, 1 \right) \times \{0\}, \left( 0, 1 \right) \times \{1\} \\
		\partial_{\mathbf{n}} u - iku & = 0 \quad \text{on} \; \{0\} \times \left( 0, 1 \right), \{1\} \times \left( 0, 1 \right) \\
		f(x, y) & = \delta(x - 0.5, y - 0.5),
	\end{split}
\end{equation}

where $\delta(x)$ is the Dirac delta function. The equation is defined on the unit square and possesses Dirichlet boundary conditions at the top and bottom and Robin radiation conditions at the left and right.

We discretize this equation on a uniform Cartesian grid using the classical five-point stencil

\begin{equation*}
	A_h = \frac{1}{h^2} \begin{bmatrix}
		& -1 & \\
		-1 & 4 - (k h)^2 & -1 \\
		& -1 &  
	\end{bmatrix}.
\end{equation*}

In addition, $\delta(x)$ is approximated with a second-order Zenger correction.
The spacing $h$ of the grid is chosen to fulfill the second-order accuracy requirement $h k = 0.625$ as described above.
Finally, we apply the shifted Laplace operator

\begin{equation*}
	M = -\nabla^{2} - (k^{2} + 0.5 i k^{2}),
\end{equation*}

as a preconditioner, which is discretized similarly to the original operator using the five-point stencil

\begin{equation*}
	M_h = \frac{1}{h^2} \begin{bmatrix}
		& -1 & \\
		-1 & 4 - (1.0 + 0.5i)(k h)^2 & -1 \\
		& -1 &  
	\end{bmatrix}.
\end{equation*}

In [2]:

cwd = f'{os.getcwd()}/..'
# Path to the ExaStencils compiler
compiler_path = f'{cwd}/exastencils/Compiler/Compiler.jar'
# Path to base folder
base_path = f'{cwd}/example_problems'
# Relative path to platform file (from base folder)
platform_path = f'lib/linux.platform'
# Example problem from L2
# Relative path to settings file (from base folder)
settings_path = f'Helmholtz/2D_FD_Helmholtz_fromL3.settings'
knowledge_path = f'Helmholtz/2D_FD_Helmholtz_fromL3.knowledge'
cycle_name = "gen_mgCycle"  # Default name
# Additional global parameter values within the PDE system
values = [80.0 * 2.0**i for i in range(100)]
pde_parameter_values = {'k': values}
# The maximum number of iterations considered acceptable for a solver
solver_iteration_limit = 10000
# Set up MPI
from mpi4py import MPI
comm = MPI.COMM_WORLD
nprocs = comm.Get_size()
mpi_rank = comm.Get_rank()

## Solver Configuration
As a result of the above formulation of our test problem, we obtain two systems of linear equations

\begin{equation}
	A_h M_h^{-1} y_h = b_h,
\end{equation}

where $b_h$ contains the values of $\delta(x)$ at each grid point, and

\begin{equation}
	M_h x_h = y_h,
\end{equation}

where $x_h$ represents the approximate solution of the above Helmholtz problem.
While for each of these two systems of linear equations, a functioning solver is needed, the focus of this example is the design of an efficient multigrid method for the approximate solution of the preconditioning system.
Here, to limit the cost of preconditioning, we assume that the application of a single multigrid cycle is sufficient to compute a reasonable approximation for $M_{h}^{-1}$.
After designing a suitable multigrid-based preconditioner, the discretized Helmholtz equation is solved using the biconjugate gradient stabilized method (BiCGSTAB).

In [3]:
program_generator = ProgramGenerator(compiler_path, base_path, settings_path, knowledge_path, platform_path, mpi_rank=mpi_rank, 
                                     cycle_name=cycle_name, model_based_estimation=False,
                                     solver_iteration_limit=solver_iteration_limit)

In [4]:
# Obtain extracted information from program generator
dimension = program_generator.dimension  # Dimensionality of the problem
finest_grid = program_generator.finest_grid  # Representation of the finest grid
coarsening_factors = program_generator.coarsening_factor
min_level = program_generator.min_level  # Minimum discretization level
max_level = program_generator.max_level  # Maximum discretization level
equations = program_generator.equations  # System of PDEs in SymPy
operators = program_generator.operators  # Discretized differential operators
fields = program_generator.fields  # Variables that occur within system of PDEs
problem_name = program_generator.problem_name


In [5]:
if mpi_rank == 0 and not os.path.exists(f'{cwd}/{problem_name}'):
    # Create directory for checkpoints and output data
    os.makedirs(f'{cwd}/{problem_name}')
# Path to directory for storing checkpoints
checkpoint_directory_path = f'{cwd}/{problem_name}/checkpoints_{mpi_rank}'

In [6]:
optimizer = Optimizer(dimension, finest_grid, coarsening_factors, min_level, max_level, equations, operators, fields, 
                      mpi_comm=comm, mpi_rank=mpi_rank, number_of_mpi_processes=nprocs, program_generator=program_generator,  
                      checkpoint_directory_path=checkpoint_directory_path)

In [7]:
levels_per_run = max_level - min_level
assert levels_per_run <= 5, "Can not optimize more than 5 levels"

In [8]:
# Choose optimization method
optimization_method = optimizer.NSGAII

generations = 20  # Number of generations
mu_ = 4  # Population size
lambda_ = 4  # Number of offspring
# Option to use random search instead of crossover and mutation to create new individuals
use_random_search = False
population_initialization_factor = 4  # Multiply mu_ by this factor to set the initial population size
crossover_probability = 0.9
mutation_probability = 1.0 - crossover_probability
node_replacement_probability = 0.1  # Probability to perform mutation by altering a single node in the tree
evaluation_samples = 3  # Number of evaluation samples

In [9]:
# Number of generations after which a generalization is performed
# This is achieved by incrementing min_level and max_level within the optimization
# Such that a larger (and potentially more difficult) instance of the same problem is considered in subsequent generations
generalization_interval = 10
# Option to continue from the checkpoint of a previous optimization
# Warning: So far no check is performed whether the checkpoint is compatible with the current optimization setting
continue_from_checkpoint = False

In [ ]:
# Return values of the optimization
# program: Grammar string representing the multigrid method on the topmost levels
# dsl_code: ExaSlang program string representing the multigrid solver functions
# pops: Populations at the end of each optimization run on the respective subrange of the discretization hierarchy
# stats: Statistics structure (data structure provided by the DEAP framework)
# hofs: Hall-of-fames at the end of each optimization run on the respective subrange of the discretization hierarchy
program, dsl_code, pops, stats, hofs = optimizer.evolutionary_optimization(optimization_method=optimization_method, 
                                                                 use_random_search=use_random_search, 
                                                                 mu_=mu_, lambda_=lambda_, 
                                                                 population_initialization_factor=population_initialization_factor,
                                                                 generations=generations, 
                                                                 generalization_interval=generalization_interval,
                                                                 crossover_probability=crossover_probability,
                                                                 mutation_probability=mutation_probability,
                                                                 node_replacement_probability=node_replacement_probability,
                                                                 levels_per_run=levels_per_run,
                                                                 evaluation_samples=evaluation_samples,
                                                                 pde_parameter_values=pde_parameter_values,
                                                                 continue_from_checkpoint=continue_from_checkpoint)

In [ ]:
# Print the outcome of the optimization and store the data and statistics
if mpi_rank == 0:
    print(f'\nExaSlang Code:\n{dsl_code}\n', flush=True)
    if not os.path.exists(f'./{problem_name}'):
        os.makedirs(f'./{problem_name}')
    j = 0
    log_dir_name = f'./{problem_name}/data_{j}'
    while os.path.exists(log_dir_name):
        j += 1
        log_dir_name = f'./{problem_name}/data_{j}'
    os.makedirs(log_dir_name)
    for i, log in enumerate(stats):
        optimizer.dump_data_structure(log, f"{log_dir_name}/log_{i}.p")
    for i, pop in enumerate(pops):
        optimizer.dump_data_structure(pop, f"{log_dir_name}/pop_{i}.p")
    for i, hof in enumerate(hofs):
        hof_dir = f'{log_dir_name}/hof_{i}'
        os.makedirs(hof_dir)
        for j, ind in enumerate(hof):
            with open(f'{hof_dir}/individual_{j}.txt', 'w') as grammar_file:
                grammar_file.write(str(ind) + '\n')